In [207]:
from xml.dom.minidom import parseString
from xml.dom import Node
from typing import Any
from functools import cmp_to_key

In [208]:
data = """
<root>
<c>
    <b/>
</c>
<d/>
<c>
    <x/>
</c>
<a>
    foobar
</a>
<!-- Comment -->
<c/>
<b/>
<c>
    <a/>
</c>
<e name="456"/>
<e/>
<e name="123"/>
</root>
"""

In [209]:
tree = parseString(data)

In [210]:
def node_less(l, r):
    # Sort nodes in this order:
    # 1. by name
    # 2. by comparing the (attribute name, attribute value) pairs
    # 3. by comparing the number of child nodes
    # 4. by comparing the children recursively until the first difference is found
    #
    # if nothing is found, the nodes compare equal
    if l.nodeName < r.nodeName:
        return -1
    elif l.nodeName > r.nodeName:
        return 1
    elif extract_attributes(l) < extract_attributes(r):
        return -1
    elif extract_attributes(l) > extract_attributes(r):
        return 1
    elif len(l.childNodes) < len(r.childNodes):
        return -1
    elif len(l.childNodes) > len(r.childNodes):
        return 1
    for i in range(min(len(l.childNodes), len(r.childNodes))):
        c = node_less(l.childNodes[i], r.childNodes[i])
        if c != 0:
            return c

    return 0


def extract_attributes(tree) -> Any:
    res = []
    if tree.attributes is not None:
        for i in range(tree.attributes.length):
            attr = tree.attributes.item(i)
            res.append((attr.name, attr.value))
    return res

def sort_nodes(tree) -> Any:
    L = []
    tree.normalize()
    for child in tree.childNodes:
        # Remove whitespace
        if child.nodeType == Node.TEXT_NODE:
            child.nodeValue = child.nodeValue.strip()
            if child.nodeValue == "":
                continue
        elif child.nodeType in (Node.COMMENT_NODE, Node.PROCESSING_INSTRUCTION_NODE):
            continue
        L.append(sort_nodes(child))

    L.sort(key=cmp_to_key(node_less))

    if len(L) > 0:
        tree.childNodes[:] = L
    return tree